# 向量模型

向量模型 (Vector Model)，又称 嵌入模型 (Embedding Model)，它负责将非结构化的文本数据（词语、句子、段落、文档）转化为 固定长度的稠密向量，从而将文本信息映射到高维的语义向量空间。

向量模型的核心作用：

- 语义表示：捕捉文本的语义信息和上下文关系，将自然语言转化为机器可理解和计算的数学表示形式——向量。语义相似的文本在向量空间中距离更近，为语义相似度计算和信息检索提供基础。

向量模型的工作原理通常基于深度学习技术，特别是 Transformer 架构 和 预训练语言模型。其核心步骤包括：

- 文本预处理：对输入文本进行分词 (tokenization)、去除停用词、小写转换等预处理操作。
- 特征编码：利用预训练语言模型（如 BERT, RoBERTa, Sentence-BERT, OpenAI Embeddings 等）对预处理后的文本进行编码，提取文本的上下文语义特征。
- 向量映射：将提取的语义特征映射到固定维度的向量空间。不同的模型采用不同的映射策略，例如 BERT 通常取 [CLS] token 的输出向量，Sentence-BERT 则采用均值池化 (mean pooling) 等策略。
- 向量归一化（可选）：对生成的向量进行归一化处理，例如 L2 归一化，使得向量长度统一，便于计算余弦相似度等。


关键技术与类型
- 词嵌入模型 (Word Embedding Models)：Word2Vec, GloVe, FastText 等，为单个词语生成向量表示，但缺乏上下文语义理解能力，在 RAG 系统中应用较少。
- 句子编码器 (Sentence Encoders)：Sentence-BERT, Universal Sentence Encoder 等，直接生成句子或段落的向量表示，更适合 RAG 系统。
- 文本嵌入模型 API：OpenAI Embeddings (text-embedding-ada-002), Cohere Embeddings, Azure OpenAI Embeddings 等，通过 API 接口提供文本向量化服务，易于使用，性能强大。





## 向量化模型如何选型？

一、选型决策要素（先圈定范围）
- 语种与数据类型：中文/英文/多语；文本/代码/表格混合
- 部署与合规：API（SaaS）还是私有化自建
- 成本与时延：目标 P95、QPS、预算；向量维度越高=索引成本越高
- 规模：文档量级、是否需压缩（PQ/OPQ）
- 领域性：是否有领域样本可微调；是否需重排器或多向量（ColBERT）
- 检索策略：纯向量 vs Hybrid（BM25+向量）；是否做查询改写

二、模型选型（按场景）
- 中文：例如bge-base-zh-v1.5（768d)
- 英文：bge-large-en-v1..5(1024d)
- 多语种：bge-m3、E5、qwen-embedding

三、快速评测（小而快）

数据：≥50–200 条带标注/半标注查询，
指标：Recall@50、nDCG@10、MRR@10、P95 延迟、QPS、索引大小/成本
流程：同一切块与索引，比较 BM25/向量/Hybrid ± 重排；选 Recall 高者，再看排序与时延；最后做小流量 A/B

## RAG索引优化？
索引：从源中获取数据并对其进行向量化索引。
检索和生成：运行时接受用户查询转化成向量化数据，并从索引中检索相关数据，然后将其传递给LLM，生成内容。

**多层索引**
多层索引技术主要是针对海量的文档检索的处理策略，对面临需要从海量的文档中搜索、找到相关信息，并将这类信息综合成一个答案，并应用其来源。一般有2种方式建立多层索引来优化这类问题。

一种是为文档建立多级索引，比如建立文档的类别标签，在处理指定业务场景下的问答时，只需要对该类别下的知识库进行检索。

第二种是从两个索引——一个是有该文档的摘要组成，一个由文档切片组成。在处理学术论文时，这类方法一般有较好的表现。这类方法的思路是分为两个步骤进行检索，首先通过摘要部分进行召回检索，过滤出可能和当前问题相关的候选文档集，然后再利用切片索引，从候选的文档集中进行检索，召回相关的知识切片。
